<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Alberto Vavassori</b> (alberto.vavassori@polimi.it) - 2023 - Politecnico di Milano, Italy <br>
</div>

## Netatmo temperature time series cleaning (part 4)

This notebook displays the Pearson's correlation coefficient between each Neatmo station and the virtual ARPA station for a selected year and month.

In [ ]:
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
import datetime
from shapely.geometry import Point

In [ ]:
year_w = widgets.Dropdown(
    options = [i for i in range(2014, 2024)],
    value = 2023,
    description = 'Year:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
year_w

In [ ]:
year = year_w.value

In [ ]:
month_w = widgets.Dropdown(
    options = [i for i in range(1, 13)],
    value = 1,
    description = 'Month:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
month_w

In [ ]:
month = month_w.value

In [ ]:
cmm_gdf = gpd.read_file('CMM.gpkg')

In [ ]:
netatmo_path = 'Netatmo_csv_files/'

In [ ]:
perason_corr = pd.read_csv(netatmo_path + 'corr_ARPA_netatmo_%s-%s.csv' %(year, month), skiprows=0)

In [ ]:
perason_corr.head()

In [ ]:
perason_corr['geometry'] = perason_corr.apply(lambda row: Point(row['long'], row['lat']), axis = 1)
perason_corr_gdf = gpd.GeoDataFrame(perason_corr, geometry = 'geometry', crs = 'EPSG:4326')

In [ ]:
m = cmm_gdf.explore(
     #style_kwds = {'fillOpacity': 0},
     marker_kwds=dict(radius=10, fill=True), # make marker radius 10px with fill
     tooltip_kwds=dict(labels=False), # do not show column label in the tooltip
     tooltip = False, 
     popup = False,
     highlight = False,
     name="cmm" # name of the layer in the map
 )

perason_corr_gdf.explore(m=m, 
     column="pearson_coef", # make choropleth based on "BoroName" column
     tooltip="pearson_coef", # show "BoroName" value in tooltip (on hover)
     #scheme="naturalbreaks",
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     marker_kwds = dict(radius = 4, fill = True),
     cmap = 'hot'
    )

m